In [28]:
import pandas as pd
import requests
import datetime
from dotenv import load_dotenv
import os


load_dotenv()
ow_key = os.getenv('ow_key')

In [29]:
df_ip = pd.read_csv('../data/inverter_plants.csv')

# Fix NaN plant_address values to 'Gazipur' for consistency
df_ip['plant_address'] = df_ip['plant_address'].fillna('Gazipur')

df_ip['plant_number'] = df_ip.index + 1

df = df_ip[['plant_id', 'plant_name', 'plant_address', 'plant_capacity', 'plant_number']]

df

,plant_id,plant_name,plant_address,plant_capacity,plant_number
0,15848107,DGL (GGI),Gazipur,1280.40,1
1,1535729,SOLshare LM Tower,Dhaka,4.25,2
2,1535728,BHC,Dhaka,15.66,3
3,11811249,ACCL (GGI),Gazipur,285.12,4
4,11605154,NAL (GGI),Gazipur,227.04,5
5,12018942,ACML (GGI),Gazipur,525.36,6
6,11701955,CAL (GGI),Gazipur,522.72,7
7,11838318,HKL (GGI),Gazipur,269.28,8
8,1535726,Unilever Sales Depot Bogura,Bogura,39.60,9
9,9977156,KCL (SOLshare),Narayangang,249.20,10


In [30]:
df_day = pd.read_csv('../data/inverter_daily_generation_logs.csv')
df_day = df_day.merge(df, on='plant_id', how='left')

plant_daylist = df_day['plant_name'].unique().tolist()
adress_daylist = df_day['plant_address'].unique().tolist()

df_day['generation_date'].min(), df_day['generation_date'].max(), adress_daylist

('2022-02-11 08:00:00.000000 UTC',
 '2025-10-22 18:00:00.000000 UTC',
 ['Gazipur', 'Rajshahi', 'Bogura', 'Dhaka', 'Narayangang'])

In [31]:
for i in adress_daylist:
    df_temp = df_day[df_day['plant_address'] == str(i)]
    start = df_temp['generation_date'].min()
    end = df_temp['generation_date'].max()
    address = df_temp['plant_name'].unique()
    # Fix: convert 'start' and 'end' to datetime first for subtraction, then get the number of days
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)
    days = (end_dt - start_dt).days
    data_point = len(df_temp)
    print(i, address)
    print(f'days = {days}')
    print(f'data_point = {data_point}')
    print(f"data_point_by_plant = {data_point/len(df_temp['plant_name'].unique())}")
    print(start)
    print(end)
    print('\n')

Gazipur ['ACCL (GGI)' 'ACML (GGI)' 'CAL (GGI)' 'DGL (GGI)' 'HKL (GGI)' 'NAL (GGI)']
days = 701
data_point = 3132
data_point_by_plant = 522.0
2023-11-21 16:51:09.000000 UTC
2025-10-22 18:00:00.000000 UTC


Rajshahi ['Faruk Auto Garage Rajshahi']
days = 647
data_point = 583
data_point_by_plant = 583.0
2022-02-11 08:00:00.000000 UTC
2023-11-21 05:33:26.000000 UTC


Bogura ['Unilever Sales Depot Bogura']
days = 402
data_point = 328
data_point_by_plant = 328.0
2022-10-14 08:00:00.000000 UTC
2023-11-20 17:55:26.000000 UTC


Dhaka ['BHC' 'SOLshare LM Tower']
days = 684
data_point = 572
data_point_by_plant = 286.0
2022-11-15 08:00:00.000000 UTC
2024-09-29 17:00:44.000000 UTC


Narayangang ['KCL (SOLshare)']
days = 679
data_point = 1407
data_point_by_plant = 1407.0
2022-11-12 08:00:00.000000 UTC
2024-09-21 18:00:00.000000 UTC




In [32]:
df_gazi = df_day[df_day['plant_address'] == 'Gazipur']

df_gazi.value_counts('generation_date').head(50)
#df_gazi[df_gazi['generation_date'] == '2024-02-19 18:00:00.000000 UTC']

generation_date
2024-09-12 18:00:00.000000 UTC    25
2024-02-19 18:00:00.000000 UTC    18
2025-10-22 18:00:00.000000 UTC     6
2025-07-31 18:00:00.000000 UTC     6
2025-07-07 18:00:00.000000 UTC     6
2025-07-08 18:00:00.000000 UTC     6
2025-07-09 18:00:00.000000 UTC     6
2025-07-10 18:00:00.000000 UTC     6
2024-02-22 18:00:00.000000 UTC     6
2025-07-11 18:00:00.000000 UTC     6
2024-02-20 18:00:00.000000 UTC     6
2024-02-18 18:00:00.000000 UTC     6
2024-02-17 18:00:00.000000 UTC     6
2024-02-15 18:00:00.000000 UTC     6
2024-02-14 18:00:00.000000 UTC     6
2024-02-13 18:00:00.000000 UTC     6
2025-07-16 18:00:00.000000 UTC     6
2025-07-17 18:00:00.000000 UTC     6
2025-07-18 18:00:00.000000 UTC     6
2025-07-19 18:00:00.000000 UTC     6
2025-07-20 18:00:00.000000 UTC     6
2025-07-21 18:00:00.000000 UTC     6
2025-07-22 18:00:00.000000 UTC     6
2025-07-23 18:00:00.000000 UTC     6
2025-07-26 18:00:00.000000 UTC     6
2025-07-28 18:00:00.000000 UTC     6
2025-07-29 18:00:00.00

In [33]:
df_weather = pd.read_csv('../data/weather_station_data_logs.csv')
df_weather = df_weather.merge(df, on='plant_id', how='left')
weather_plant = df_weather['plant_name'].unique().tolist()
weather_plant

['NAL (GGI)', 'ACML (GGI)', 'CAL (GGI)', 'HKL (GGI)']

In [34]:
df_weather.value_counts('plant_name')

plant_name
NAL (GGI)     24599
CAL (GGI)     14631
HKL (GGI)     14240
ACML (GGI)    10869
dtype: int64

In [35]:
df_weather.sort_values('timestamp')
df_weather['daystamp'] = pd.to_datetime(df_weather['timestamp']).dt.date
df_weather.groupby('daystamp').agg({
    'weather_data_wm2t': 'mean',
    'weather_data_tamb': 'mean',
    'weather_data_tmod': 'mean'
})

,weather_data_wm2t,weather_data_tamb,weather_data_tmod
daystamp,,,
2025-02-09,0.000000,14.609500,18.043250
2025-02-10,271.421172,22.741701,29.182993
2025-02-11,210.753931,23.289007,28.493910
2025-02-12,229.402299,23.305183,28.709863
2025-02-13,240.192627,25.075826,30.681232
...,...,...,...
2025-10-23,197.909134,17.093347,35.788763
2025-10-24,237.129404,16.629395,36.304244
2025-10-25,214.933727,16.904288,36.123243


In [36]:
# 'DGL (GGI)' : [15959518, 15848112, 15959516, 15848111]
# 'NAL (GGI)' : [11811369, 11811476, 12635561]
# 'ACML (GGI)' : [12667024, 12019165]
# 'ACCL (GGI)' : [15834240, 15834239]
# 'HKL (GGI)' : [16008274]
# 'CAL (GGI)' : [11848078]

In [37]:
# Remove " , " from the value in generation_amount :
df_temp['generation_amount'] = df_temp['generation_amount'].astype(str).str.replace(',', '')

# Transform "generation_amount" value to float
df_temp['generation_amount'] = df_temp['generation_amount'].astype(float)

df_temp = df_day[df_day['plant_address'] == 'Gazipur']
df_temp.sort_values('generation_amount').tail(20)

# generation amout is KWhour
# plant_capacity

/var/folders/2c/jlpbrr154jg9cfp1pzbh8x5h0000gn/T/ipykernel_92291/2441851925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['generation_amount'] = df_temp['generation_amount'].astype(str).str.replace(',', '')
/var/folders/2c/jlpbrr154jg9cfp1pzbh8x5h0000gn/T/ipykernel_92291/2441851925.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['generation_amount'] = df_temp['generation_amount'].astype(float)


,plant_id,generation_date,generation_amount,plant_name,plant_address,plant_capacity,plant_number
5782,11838318,2025-06-13 18:00:00.000000 UTC,995,HKL (GGI),Gazipur,269.28,8
4204,11838318,2024-07-14 18:00:00.000000 UTC,995.4,HKL (GGI),Gazipur,269.28,8
2774,11605154,2024-03-31 18:00:00.000000 UTC,995.477,NAL (GGI),Gazipur,227.04,5
409,11838318,2025-07-06 18:00:00.000000 UTC,995.6,HKL (GGI),Gazipur,269.28,8
5448,11605154,2025-10-20 18:00:00.000000 UTC,996.03,NAL (GGI),Gazipur,227.04,5
62,11701955,2025-09-22 18:00:00.000000 UTC,996.049,CAL (GGI),Gazipur,522.72,7
4934,11838318,2025-01-21 18:00:00.000000 UTC,996.7,HKL (GGI),Gazipur,269.28,8
5901,11605154,2025-04-13 18:00:00.000000 UTC,996.865,NAL (GGI),Gazipur,227.04,5
5112,11701955,2024-12-14 18:00:00.000000 UTC,996.9,CAL (GGI),Gazipur,522.72,7
5041,11838318,2024-10-30 18:00:00.000000 UTC,997.1,HKL (GGI),Gazipur,269.28,8


In [38]:
df = pd.read_csv('../data/plants_billing_meter_logs.csv')
df['meter_id'].value_counts()

12019165    405
12667024    405
11811476    403
11811369    403
12635561    403
15834240    398
15834239    398
11848078    398
11838324    346
15959518    107
15848112    107
15959516    107
15848111    107
16008274     98
14894877     63
Name: meter_id, dtype: int64

In [39]:
# Gazipur = 2023-11-21, 2025-10-22
# Dhaka = 2022-11-15, 2024-09-29
# Bogura = 2022-10-14, 2023-11-20
# Narayangang = 2022-11-12, 2024-09-21
# Rajshahi = 2022-02-11, 2023-11-21

In [40]:
# df_hour = pd.read_csv('../data/inverter_hourly_generation_logs.csv')
# df_hour = df_hour.merge(df, on='plant_id', how='left')

# plant_daylist = df_hour['plant_name'].unique().tolist()
# adress_daylist = df_hour['plant_address'].unique().tolist()

# df_hour['generation_date'].min(), df_hour['generation_date'].max(), adress_daylist


In [41]:
# for i in adress_daylist:
#     df_temp = df_hour[df_hour['plant_address'] == str(i)]
#     start = df_temp['generation_date'].min()
#     end = df_temp['generation_date'].max()
#     address = df_temp['plant_name'].unique()
#     # Fix: convert 'start' and 'end' to datetime first for subtraction, then get the number of days
#     start_dt = pd.to_datetime(start)
#     end_dt = pd.to_datetime(end)
#     days = (end_dt - start_dt).days
#     data_point = len(df_temp)
#     print(i, address)
#     print(f'days = {days}')
#     print(f'data_point = {data_point}')
#     print(f"data_point_by_plant = {data_point/len(df_temp['plant_name'].unique())}")
#     print(start)
#     print(end)
#     print('\n')

In [43]:
df_5m = pd.read_csv('../data/inverter_five_minutes_generation_logs.csv')
df_5m = df_5m.merge(df, on='plant_id', how='left')

plant_daylist = df_5m['plant_name'].unique().tolist()
adress_daylist = df_5m['plant_address'].unique().tolist()

df_5m['generation_date'].min(), df_5m['generation_date'].max(), adress_daylist

: 

In [ ]:
for i in adress_daylist:
    df_temp = df_5m[df_5m['plant_address'] == str(i)]
    start = df_temp['generation_date'].min()
    end = df_temp['generation_date'].max()
    address = df_temp['plant_name'].unique()
    print(i, address)
    print(start)
    print(end)
    print('\n')

In [ ]:
url = f"https://api.openweathermap.org/data/2.5/weather?q=Gazipur,BD&appid={ow_key}}"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Erreur : {response.status_code}")


# To access the temp value from the data dictionary:
temp_value = data['main']['temp']
cloud = data['clouds']['all']
visibility = data['visibility']

# Convert the UNIX timestamp to a readable date and time (UTC)
sunrise = datetime.datetime.utcfromtimestamp(data['sys']['sunrise'])
sunset = datetime.datetime.utcfromtimestamp(data['sys']['sunset'])
sun_time = (sunset - sunrise).total_seconds()


print(f" temp : {temp_value}, cloud : {cloud}, visibility : {visibility}, sunset : {sunset}, sunrise : {sunrise}, sun_time : {sun_time}")
print(data)

In [ ]:
# # To access the temp value from the data dictionary:
# temp_value = data['main']['temp']
# cloud = data['clouds']['all']
# visibility = data['visibility']

# # Convert the UNIX timestamp to a readable date and time (UTC)
# sunrise = datetime.datetime.utcfromtimestamp(data['sys']['sunrise'])
# sunset = datetime.datetime.utcfromtimestamp(data['sys']['sunset'])
# sun_time = (sunset - sunrise).total_seconds()

# print(temp_value, cloud, visibility, sunset, sunrise, sun_time)


In [ ]:
weather_df = pd.read_csv('data/weather_station_data_logs.csv')
weather_df.sort_values('timestamp')

In [ ]:
from weather_retrivial import weather_retrivial

url = f"https://api.openweathermap.org/data/2.5/weather?q=Gazipur,BD&appid={ow_key}}"

weather_retrivial(url)

In [ ]:
df_weather = pd.read_csv('data/weather_station_data_logs.csv')
df_weather = df_weather.merge(df, on='plant_id', how='left')
df_weather

In [ ]:
# Watt per m2 WM2t
# tamb ambient
# tmod module temp (surface temp of panel)


NOT ACTUAL BUT EXPECTED FOR THIS WEATHER CONDITION